##   BLM442E Spring 2020
## Homework 1


## Question 1
Generate an RSA public-private key pair. K A+ and K A- .

In [1]:
import random
from sympy import mod_inverse

'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    #Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    print("e =",e)
    d = mod_inverse(e,phi)
    print("d =",d)
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

def encrypt(pk, plaintext):
   
    #Unpack the key into it's components
    e, n = pk
     #c = m ^e mod  n
    #Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [(ord(char) ** e) % n for char in plaintext]
    #Return the array of bytes
    return cipher

def decrypt(pk, ciphertext):
    #Unpack the key into its components
    d, n = pk
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr((char ** d) % n) for char in ciphertext]
    #Return the array of bytes as a string
    return ''.join(plain)


In [2]:
# in real word primes must be very very big
primes =list(filter(lambda x: is_prime(x),list(range(100, 199))))
p = random.choice(primes) 
#to prevent seleccting same prime we delete it from the list
primes.remove(p)
q=random.choice(primes) 
public, private = generate_keypair(p, q)
print("p =",p)
print("q =",q)
print ("Your public key is ", public ," and your private key is ", private)

e = 2971
d = 11275
p = 103
q = 149
Your public key is  (2971, 15347)  and your private key is  (11275, 15347)


## Question 2
Generate two symmetric keys: 128 bit K 1 and 256 bit K 2 . Print values of the keys on the
screen. Encypt them with K A+ , print the results, and then decrypt them with K A- . Again print the
results. Provide a screenshot showing your results.

In [3]:
k1=random.randrange(2**127, 2**128)
k2=random.randrange(2**255, 2**256)
print("k1 (",k1.bit_length()," bit) = ",k1)
print("k2 (",k2.bit_length()," bit) = ",k2)

k1 ( 128  bit) =  327416971492181572621324843237217013296
k2 ( 256  bit) =  104172059098179262338679915562060991909540178289310794867454593287495395101302


In [4]:
from Crypto.Random import get_random_bytes
k1=get_random_bytes(16)
k2=get_random_bytes(32)
print("k1  = ",k1)
print("k2  = ",k2)

k1  =  b'\x14Q\x1c\x91@H=\xc2\x16\xdf\x9e6\xac\xa8,\xf1'
k2  =  b'\x84g\xa3pQ\x1dh\xda\x8d\xfb\x00m\x88\tV\xf1\\tei\x01i1\xc9\x9cS3\xb4\xac\xe3\xc4`'


In [5]:

encrypted_msg = encrypt(public, str(k1))
print ("Your encrypted k1 is: ")
print (''.join(map(lambda x: str(x), encrypted_msg)))
print ("Decrypting k1 with private key ", private )
print ("Your k1 is:")
print (decrypt( private, encrypted_msg))

Your encrypted k1 is: 
120871366148903378981012066772214890337898101448514890337885039810800713914112911489033781448587591489033789810965714890337886669475148903378850311893965714890337815214485148903378152112839641148903378947598101366
Decrypting k1 with private key  (11275, 15347)
Your k1 is:
b'\x14Q\x1c\x91@H=\xc2\x16\xdf\x9e6\xac\xa8,\xf1'


In [6]:
encrypted_msg = encrypt(public, str(k2))
print ("Your encrypted k2 is: ")
print (''.join(map(lambda x: str(x), encrypted_msg)))
print ("Decrypting k2 with private key ", private )

print ("Your k2 is:")
print (decrypt(private , encrypted_msg))

Your encrypted k2 is: 
120871366148903378112831206638111489033781529667915777221489033789810866687561489033788666152148903378112838666148903378947512087148903378151715171169314890337811283112831489043028158148903378947598101489014890430211893150931489033781517981015093981014890337814485850314890337885031448541499667148903378120871206614890337815214485148903378118939667148903378144851206673181366
Decrypting k2 with private key  (11275, 15347)
Your k2 is:
b'\x84g\xa3pQ\x1dh\xda\x8d\xfb\x00m\x88\tV\xf1\\tei\x01i1\xc9\x9cS3\xb4\xac\xe3\xc4`'


## Question 3
Consider a long text m. Apply SHA256 Hash algorithm (Obtain the message digest, H(m)).
Then encrypt it with K A- . (Thus generate a digital signature.) Then verify the digital signature.
(Decrypt it with K A+ , apply Hash algorithm to the message, compare). Print m, H(m) and digital
signature on the screen. Provide a screenshot. (Or you may print in a file and provide the file)


### Generating a digital signature

* for  creating digital signature we need hashed message and then encrypt it with private key digital signature= (message, K^-1(h(message)))

In [7]:
from hashlib import sha256
def hashFunction(message):
    hashed = sha256(message.encode("UTF-8")).hexdigest()
    return hashed
message="this secret  message is writen by maya kara is here"
hMsg= hashFunction(message)
encryptedHMsg=encrypt(private, hMsg)
print ("- your message is: ")
print(message)
print ("- Your encrypted H(message) is : ")
print (''.join(map(lambda x: str(x), encryptedHMsg)))

- your message is: 
this secret  message is writen by maya kara is here
- Your encrypted H(message) is : 
36501303526659728761237126623719642126535972674836501303413034964214929123381233836505355266365052213035130358761365012653266365013035237126151233814929261526623712118535513035365012338237196429642130351303521182371211814929211821182615126532371126536748130351233859722615


###  Verifying a digital signature
* for verify the digital signature we need to Decrypt the encrypted H(m) in signature  with K A+ to obtain H(m)
* get H(m) of the pure message in signature
* compare bothe H(m) values if they are equal thr signature is valid

In [8]:
print ("First  H(m) value: ",decrypt(public , encryptedHMsg))
print ("Second H(m) value: ",hashFunction(message))

First  H(m) value:  39a067a72e0c38821dd3fa3b9963ea3975d15a74f93d722994741445e7ec9d05
Second H(m) value:  39a067a72e0c38821dd3fa3b9963ea3975d15a74f93d722994741445e7ec9d05


## Question 4
Generate or find any file of size 1MB. Now consider following three algorithms:
* i) AES (128 bit key) in CBC mode.
* ii) AES (256 bit key) in CBC mode.
* iii) DES in CBC mode (you need to generate a 56 bit key for this).
* a) Encrypt the file of size 1MB. Store the result (and submit it with the homework) (Note: IV
should be randomly generated, Key = K 1 or K 2 ).
* b) Decrypt the file and store the result. Show that it is the same as the original file.
* c) Measure the time elapsed for encryption. Write it in your report. Comment on the result.
* d) For the first algorithm, change Initialization Vector (IV) and show that the corresponding
ciphertext chages for the same plaintext (Give the result for both).

In [9]:

from Crypto import Random
from Crypto.Cipher import AES
import os
import os.path
from os import listdir
from os.path import isfile, join
import time


class Encryptor:
    def __init__(self, key):
        self.key = key
    # for soving not completed block in file end problem    
    def pad(self, s):
        return s + b"\0" * (AES.block_size - len(s) % AES.block_size)

    def encrypt(self, message, key, key_size=128):
        message = self.pad(message)
        iv = Random.new().read(AES.block_size)
        print("iv= ",iv)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        return iv + cipher.encrypt(message)

    def encrypt_file(self, file_name):
        with open(file_name, 'rb') as fo:
            plaintext = fo.read()
        enc = self.encrypt(plaintext, self.key)
        with open(file_name + ".enc", 'wb') as fo:
            fo.write(enc)
        os.remove(file_name)
        return enc
       

    def decrypt(self, ciphertext, key):
        iv = ciphertext[:AES.block_size]
        cipher = AES.new(key, AES.MODE_CBC, iv)
        plaintext = cipher.decrypt(ciphertext[AES.block_size:])
        return plaintext.rstrip(b"\0")

    def decrypt_file(self, file_name):
        with open(file_name, 'rb') as fo:
            ciphertext = fo.read()
        dec = self.decrypt(ciphertext, self.key)
        with open(file_name[:-4], 'wb') as fo:
            fo.write(dec)
        os.remove(file_name)
        return dec
 

In [10]:
start_time = time.time()
enc = Encryptor(k1)
clear = lambda: os.system('cls')
enc.encrypt_file("data.txt")
print("encryption time with k1 ( 128  bit) =  %s seconds" % (time.time() - start_time))

start_time = time.time()
enc.decrypt_file("data.txt.enc")
print("decryption time with k1 ( 128  bit) =  %s seconds" % (time.time() - start_time))


iv=  b'o\xdaH\xe2\xbfy\x17\x8a\xc5t \xc7n;\xbf\xf5'
encryption time with k1 ( 128  bit) =  0.022490739822387695 seconds
decryption time with k1 ( 128  bit) =  0.016085147857666016 seconds


In [11]:
start_time = time.time()
enc = Encryptor(k2)
clear = lambda: os.system('cls')
enc.encrypt_file("data.txt")
print("encryption time with k2 ( 256  bit) =  %s seconds" % (time.time() - start_time))

start_time = time.time()
enc.decrypt_file("data.txt.enc")
print("decryption time with k2 ( 256  bit) =  %s seconds" % (time.time() - start_time))

iv=  b'\xeen\x07x\xd17_X\xaeY\xc7\xbb\xe75\x9e\xe7'
encryption time with k2 ( 256  bit) =  0.026482582092285156 seconds
decryption time with k2 ( 256  bit) =  0.01511383056640625 seconds


In [12]:
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

class DESencryptor:
    def __init__(self, key):
        self.key = key
        self.des =   DES.new(self.key, DES.MODE_ECB)


    def encrypt(self, message):
        message = pad(message,8)
        
        return self.des.encrypt(message)

    def encrypt_file(self, file_name):
        with open(file_name, 'rb') as fo:
            plaintext = fo.read()
        enc = self.encrypt(plaintext)
        with open(file_name + ".enc", 'wb') as fo:
            fo.write(enc)
        os.remove(file_name)
        return enc
    def decrypt(self, ciphertext):
        return unpad(self.des.decrypt(ciphertext),8)

    def decrypt_file(self, file_name):
        with open(file_name, 'rb') as fo:
            ciphertext = fo.read()
        dec = self.decrypt(ciphertext)
        with open(file_name[:-4], 'wb') as fo:
            fo.write(dec)
        os.remove(file_name)
        return dec
       




key = get_random_bytes(8)

start_time = time.time()
desEnc = DESencryptor(key)
clear = lambda: os.system('cls')
desEnc.encrypt_file("data2.txt")
print("encryption time = %s seconds ---" % (time.time() - start_time))
start_time = time.time()
desEnc.decrypt_file("data2.txt.enc")
print("decryption time = %s seconds ---" % (time.time() - start_time))

encryption time = 0.0010089874267578125 seconds ---
decryption time = 0.00044345855712890625 seconds ---
